# Предобработка данных

In [56]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
df = pd.read_csv('zomatocleaned.csv')
df.head()

,Unnamed: 0,name,online_order,book_table,rating,votes,location,rest_type,dish_liked,cuisines,approx_cost,type
0,0,Jalsa,Yes,Yes,4.1,775,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,Buffet
1,1,Spice Elephant,Yes,No,4.1,787,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,Buffet
2,2,San Churro Cafe,Yes,No,3.8,918,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,Buffet
3,3,Addhuri Udupi Bhojana,No,No,3.7,88,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,Buffet
4,4,Grand Village,No,No,3.8,166,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,Buffet


Для наших целей нет разницы какого типа ресторан (rest_type, type), какова приблизительная стоимость заказа, а также есть ли в нем возможность онлайн заказа и бронирования столика.

In [57]:
dropcols = ['Unnamed: 0', 'online_order', 'book_table', 'rest_type', 'approx_cost', 'type']
df = df.drop(columns=dropcols)
# Теперь удалим дупликаты строк, различавшихся только по удаленным столбцам:
df = df.drop_duplicates()

# Любимое блюдо

Естественно определить "любимое блюдо" как блюдо, встречающееся наибольшее количество раз среди dish_liked.

In [58]:
dishes = df['dish_liked'].str.split(', ').explode()
dishes_count = dishes.value_counts()
dishes_count

Pasta                  1406
Cocktails              1306
Burgers                1231
Pizza                  1194
Biryani                 944
                       ... 
Mango Dal                 1
Pasta Barbaresca          1
Uddin Vada                1
Oregano Chicken           1
Vegetable Dumplings       1
Name: dish_liked, Length: 2790, dtype: int64

**Таким образом, любимым блюдом в нашей выборке оказалось Pasta.**

# Лучшее место

Чтобы выбрать лучшее место, хочется использовать и rating и votes, но перемножать плохо, так как, например, rating(1) * 5000 не есть хорошее место,\
Поэтому я предложу метрику "устойчивость" = (rating * (votes) + 1) / (votes+1). Данная метрика показывает какой станет рейтинг, если ресторану поставят дополнительно рейтинг 1.

In [60]:
df['stability'] = (df['rating']*df['votes'] + 1) / (df['votes'] + 1)

In [61]:
restaurants_with_Pasta = df[df['dish_liked'].str.contains('Pasta', na = False, regex=False)]
restaurants_with_Pasta.sort_values(by='stability', ascending=False)

,name,rating,votes,location,dish_liked,cuisines,stability
29846,Flechazo,4.9,3004,Whitefield,"Pizza, Mutton Kebab, Chicken Biryani, Sea Food...","Asian, Mediterranean, North Indian, BBQ",4.898702
5155,Flechazo,4.9,2745,Whitefield,"Pizza, Mutton Kebab, Chicken Biryani, Sea Food...","Asian, Mediterranean, North Indian, BBQ",4.898580
14023,Barbeque Nation,4.8,3864,JP Nagar,"Lunch Buffet, Goan Fish Curry, Mocktails, Chic...","North Indian, European, Mediterranean, BBQ, Kebab",4.799017
20167,Biergarten,4.8,498,Koramangala 5th Block,"Penne Arrabiata Pasta, Craft Beer, Bruschettas...","Continental, North Indian, Chinese, European, ...",4.792385
19596,Biergarten,4.8,482,Koramangala 5th Block,"Penne Arrabiata Pasta, Craft Beer, Bruschettas...","Continental, North Indian, Chinese, European, ...",4.792133
...,...,...,...,...,...,...,...
26353,The Twisted Desi,0.0,0,Indiranagar,"Vegetable Biryani, Dal Makhani, Matar Methi Ma...","Continental, Italian, North Indian, Chinese",1.000000
8616,Biergarten,0.0,0,Koramangala 5th Block,"Beer, Mezze Platter, Penne Arrabiata Pasta, Br...","Continental, North Indian, Chinese, European, ...",1.000000
8175,Biergarten,0.0,0,Koramangala 5th Block,"Beer, Mezze Platter, Penne Arrabiata Pasta, Co...","Continental, North Indian, Chinese, European, ...",1.000000
24963,The Twisted Desi,0.0,0,Indiranagar,"Vegetable Biryani, Dal Makhani, Matar Methi Ma...","Continental, Italian, North Indian, Chinese",1.000000


Могло получиться так, что мы выбрали лучший ресторан, но по факту он оказался лучшим из-за хинкалей, поэтому ограничим выбор среди тех, которые имеют итальянскую кухню:

In [66]:
restaurants_with_Pasta_Italy = restaurants_with_Pasta[restaurants_with_Pasta['cuisines'].str.contains('Italian', na = False, regex=False)]
restaurants_with_Pasta_Italy.sort_values(by='stability', ascending=False)

,name,rating,votes,location,dish_liked,cuisines,stability
20909,Chianti,4.7,4872,Koramangala 5th Block,"Pasta, Tiramisu, Wine, Barbeque Chicken Pizza,...",Italian,4.699241
20183,Chianti,4.7,4870,Koramangala 5th Block,"Pasta, Tiramisu, Wine, Barbeque Chicken Pizza,...",Italian,4.699240
19405,Chianti,4.7,4860,Koramangala 5th Block,"Pasta, Tiramisu, Wine, Barbeque Chicken Pizza,...",Italian,4.699239
17351,Chianti,4.7,4857,Koramangala 5th Block,"Pasta, Tiramisu, Wine, Barbeque Chicken Pizza,...",Italian,4.699238
8186,Chianti,4.7,4845,Koramangala 5th Block,"Pasta, Tiramisu, Wine, Barbeque Chicken Pizza,...",Italian,4.699236
...,...,...,...,...,...,...,...
1628,Calvin's,2.9,231,JP Nagar,"Pasta, Sandwiches, Club Sandwich, Cottage Chee...","Desserts, Italian, Pizza",2.891810
26353,The Twisted Desi,0.0,0,Indiranagar,"Vegetable Biryani, Dal Makhani, Matar Methi Ma...","Continental, Italian, North Indian, Chinese",1.000000
2216,Thyme & Whisk,0.0,0,Jayanagar,"Pizza, Momos, Brownie, Pasta, Nachos, Waffles,...","Asian, Chinese, Continental, Italian",1.000000
24963,The Twisted Desi,0.0,0,Indiranagar,"Vegetable Biryani, Dal Makhani, Matar Methi Ma...","Continental, Italian, North Indian, Chinese",1.000000


**Таким образом ресторан Chianti - лучшее место, чтобы попробовать Pasta**

# Статистический анализ

1) Я выдвинул предположение, что "любимое блюдо" определяется как блюдо, встречающееся наибольшее количество раз среди dish_liked.
2) Исходя из выбора любимого блюда (Pasta) я строю предположение, что лучший ресторан, в котором можно попробовать данное блюдо должен быть рестораном Итальянской кухни.

Естественно также рассмотреть гипотезу о связи любимого блюда с "стабильностью": "рестораны, предлагающие любимое блюдо, имеют более высокую стабильность" (стабильность можно заменить на рейтинг). То есть рассмотрим H0: средняя stability ресторанов с Pasta и без Pasta не отличаются.

Для этого предположим, что так как данных достаточно много, то по согласно ЦПТ мы можем взять стандартный t-student тест на то, что две выборки имеют одинаковое среднее значение. Напомню, что первая выборка - stability ресторанов которые содержат Pasta в понравившехся, а вторая - stability ресторанов которые не содержат Pasta.

In [79]:
from scipy import stats
# Берем средний рейтинг ресторанов с популярным блюдом
avg_stability_popular_dish = restaurants_with_Pasta['stability'].mean()

# Берем средний рейтинг остальных ресторанов
avg_stability_other = df[~df['dish_liked'].str.contains('Pasta', na=False, regex=False)]['stability'].mean()

# Проводим t-тест
t_stat, p_value = stats.ttest_ind(
    df['stability'][df['dish_liked'].str.contains('Pasta', na=False, regex=False)],
    df['stability'][~df['dish_liked'].str.contains('Pasta', na=False, regex=False)],
    nan_policy='omit'
)

print('Средняя "стабильность" ресторанов с Pasta:',   avg_stability_popular_dish)
print('Средняя "стабильность" остальных ресторанов:', avg_stability_other)
print('t-статистика:', t_stat)
print('p-значение:',  p_value)

if p_value < 0.05:
    print('Средние "стабильности" ресторанов с Pasta и остальных ресторанов статистически значимо различаются')
else:
    print('Нет достаточных доказательств отклонить нулевую гипотезу о равенстве средних стабильностей')

Средняя "стабильность" ресторанов с Pasta: 4.069482569064422
Средняя "стабильность" остальных ресторанов: 3.475760097633878
t-статистика: 30.74896557361554
p-значение: 2.3250482074696157e-202
Средние "стабильности" ресторанов с Pasta и остальных ресторанов статистически значимо различаются


Таким образом, мы действительно можем отвергнуть нулевую гипотезу о том, что средняя stability ресторанов с Pasta и без Pasta не отличаются и результат что средняя stability ресторанов с Pasta выше - статичтически значим.